In [ ]:
# Import library for consume message from kafka

import os
from dotenv import load_dotenv
from kafka import KafkaConsumer
from pathlib import Path
from proto import transaction_product_pb2

In [ ]:
# Load environment file

dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

In [ ]:
# Set kafka variable configuration

kafka_host = os.getenv('KAFKA_HOST')
kafka_host_broker_2 = os.getenv('KAFKA_BROKER_2_HOST')
kafka_topic = os.getenv('topic')
kafka_topic_partition = os.getenv('topic') + "-1"

In [ ]:
# Because topic sent protobuf formating, message need to parse to object protobuf

def parse_message(message):
    """
    parse message from string to protobuf object

    Args:
        message (string): message string
    """
    transaction = transaction_product_pb2.TransactionProduct()
    transaction.ParseFromString(message)
    return transaction

In [ ]:
# Create consume process

consumer = KafkaConsumer(kafka_topic,
                         group_id='dibimbing-assignment-group',
                         bootstrap_servers=[f'{kafka_host}:9092', f'{kafka_host_broker_2}:9095'],
                         value_deserializer=parse_message,
                         auto_offset_reset='earliest',
                         enable_auto_commit=False
                         )


In [ ]:
# Load message from kafka topic when consume and check avarage total payment whare transaction status is checkout

message_count = 0
total_payment = 0


for message in consumer:
    transaction = message.value
    if transaction.status == "checkout":
        message_count += 1
        total_payment += transaction.total_payment
        print("="*20)
        print("MESSAGE COUNT\t\t: ", message_count)
        print("TOTAL PAYMENT\t\t: ", total_payment)
        print("="*20)
        print("AVERAGE TOTAL PAYMENT\t: ", total_payment/message_count)
        print("")
        print("")
